Based on https://learn.deeplearning.ai/chatgpt-prompt-eng/

In [1]:
import openai

In [16]:
# Set API Key

openai.api_key="sk-..."
# or 
# !export OPENAI_API_KEY='sk-...'  - this did not work

In [14]:
# Code to run when using env variable for api key
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

openai.api_key  = os.getenv('OPENAI_API_KEY')

In [9]:
# Helper function for prompting
def get_completion_from_messages(messages, model="gpt-3.5-turbo", temperature=0):
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature, # this is the degree of randomness of the model's output
    )
#     print(str(response.choices[0].message))
    return response.choices[0].message["content"]

## Examples
### General Conversation

In [ ]:
messages =  [  
# Role 'system' used to provide directions to chatGPT
{'role':'system', 'content':'You are Yoda of Star Wars fame.'}, 
# Role 'user' used to specify user inputs
{'role':'user', 'content':'tell me about Skywalker'}  ]

### General Conversation with UI

In [ ]:
# Helper Function
def collect_messages(_):
    prompt = inp.value_input
    context = [{'role':'system', 'content':'You are Yoda of Star Wars fame.'}]
    # Store user inputs for context
    context.append({'role':'user', 'content':f"{prompt}"})
    response = get_completion_from_messages(context) 
    inp.value = ''
    if len(prompt.strip()) > 0:
        panels.append(
            pn.Row('User:', pn.pane.Markdown(prompt, width=500)))
    panels.append(
        pn.Row('Assistant:', pn.pane.Markdown(response, width=500, style={'background-color': '#F6F6F6'})))
 
    return pn.Column(*panels)

In [ ]:
# Config
import panel as pn  # GUI
pn.extension()

panels = [] # collect display 

inp = pn.widgets.TextInput(value="Hi", placeholder='Enter text here…')
button_conversation = pn.widgets.Button(name="Ask Bot!")

interactive_conversation = pn.bind(collect_messages, button_conversation)

dashboard = pn.Column(
    inp,
    pn.Row(button_conversation),
    pn.panel(interactive_conversation, loading_indicator=True, height=300),
)

dashboard

### Chat with Context

In [ ]:
# Helper Function
def collect_messages_with_context(_):
    prompt = inp.value_input
    # Store user inputs for context
    context.append({'role':'user', 'content':f"{prompt}"})
    response = get_completion_from_messages(context) 
    # Store responses for context
    context.append({'role':'assistant', 'content':f"{response}"})
    inp.value = ''
    if len(prompt.strip()) > 0:
        panels.append(
            pn.Row('User:', pn.pane.Markdown(prompt, width=500)))
    panels.append(
        pn.Row('Assistant:', pn.pane.Markdown(response, width=500, style={'background-color': '#F6F6F6'})))
 
    return pn.Column(*panels)

In [ ]:
# Setup
import panel as pn  # GUI
pn.extension()

panels = [] # collect display 

context = [ {'role':'system', 'content':"""
You are Bookie, an automated service to collect orders for a book store. \
You first greet the customer, then collect the order. \
You wait to collect the entire order, then summarize it and check for a final \
time if the customer wants to add anything else. \
If it's a hardcover or paperback, you ask if it is pickup or delivery. \
If it's a delivery, you ask for an address. \
Finally you collect the payment.\
Make sure to clarify the type - kindle, audiobook, hardcover, paperback to \
ensure correctness of the price.\
You respond in a short, very conversational friendly style. \
The list of available books includes \
Adventure:
ABook1  12.95(H), 10.00(A), 7.00(P), 9.00(K) \
ABook2   10.95(H), 9.25(A), 6.50(P), 5.00(K) \
ABook3   11.95(H), 9.75(A), 6.75(P), 7.00(K) \
Thrillers: \
TBook1 11.95(H), 8.75(A), 5.75(P), 5.00(K) \
TBook2 10.95(H), 7.75(A), 4.75(P), 4.00(K) \
TBook3 12.95(H), 9.75(A), 6.75(P), 8.00(K) \
Mystery: \
MBook1 11.95(H), 8.75(A), 6.75(P), 7.00(K) \
MBook2 10.95(H), 7.75(A), 3.75(P), 4.00(K) \
MBook3 12.95(H), 9.75(A), 8.75(P), 8.00(K) \
"""} ]  # accumulate messages

inp = pn.widgets.TextInput(value="Hi", placeholder='Enter text here…')
button_conversation = pn.widgets.Button(name="Send!")

interactive_conversation = pn.bind(collect_messages_with_context, button_conversation)

dashboard = pn.Column(
    inp,
    pn.Row(button_conversation),
    pn.panel(interactive_conversation, loading_indicator=True, height=300),
)

dashboard

In [ ]:
messages =  context.copy()
messages.append(
{'role':'system', 'content':'create a json summary of the previous order. Itemize the price for each item\
 The fields should be 1) title, include type 2) pickup or delivery 3) total price '},    
)
 #The fields should be 1) pizza, price 2) list of toppings 3) list of drinks, include size include price  4) list of sides include size include price, 5)total price '},    

response = get_completion_from_messages(messages, temperature=0)
print(response)